# PARSING SECTION

In [4]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
type(xls)

In [2]:
from pandas.io.excel._base import ExcelFile

ModuleNotFoundError: No module named 'pandas'

In [3]:
! pip list

Package         Version
--------------- -----------
numpy           2.2.6
pandas          2.3.0
pip             25.1.1
python-dateutil 2.9.0.post0
pytz            2025.2
setuptools      80.3.1
six             1.17.0
tzdata          2025.2


In [ ]:
#!curl -O https://www.sbimf.com/docs/default-source/scheme-factsheets/sbi-large-midcap-fund-factsheet-february-2025.pdf

In [ ]:
!curl -O https://www.sbimf.com/docs/default-source/scheme-portfolios/all-schemes-monthly-portfolio---as-on-28th-february-2025.xlsx

In [ ]:
#!curl -O https://www.sbimf.com/docs/default-source/scheme-portfolios/all-schemes-monthly-portfolio---as-on-31st-march-2025.xlsx

In [ ]:
sheet_name = 'SCF'

In [ ]:
file_path = "all-schemes-monthly-portfolio---as-on-28th-february-2025.xlsx"

# Load the workbook
xls = pd.ExcelFile(file_path)


In [ ]:
# snippet fining the fund name
# temp = pd.read_excel(file_path, sheet_name=sheet_name, nrows=5)
# temp = temp.iloc[:,[2,3]]
# fund_name = temp.iloc[1,1]
# fund_name


In [ ]:
# find_det = xls.parse(sheet_name,nrows=4)

In [ ]:
# find_det

In [ ]:
# read the data from dataframes
df = xls.parse('SCF', header=5)
df.dropna(how='all', inplace=True) #drop completly blank rows , make changes directly to the df -- inplace = true
df = df.loc[:, ~df.columns.str.contains('BRSR')]


In [ ]:
df.info()

In [ ]:
# rename the cols 

df.columns = df.columns.str.replace(r'Name of.*', 'inst_name', regex=True)
df.columns = df.columns.str.replace(r'Rating.*', 'sector', regex=True)
df.columns = df.columns.str.replace(r'Market[\s\S]*', 'market_value_lakhs', regex=True)
df.columns = df.columns.str.replace(r'YTM.*', 'ytm_percent', regex=True)
df.columns = df.columns.str.replace(r'YTC.*', 'ytc_percent', regex=True)
df.columns = df.columns.str.replace(r'Notes.*', 'notes_symbols', regex=True)
df = df.rename(columns={
     "ISIN":"isin",
     "Quantity":"quantity",
     "% to AUM":"aum_percent",
})


In [ ]:
type(df)

In [ ]:
interested_cols=['inst_name','isin','sector','quantity','market_value_lakhs','aum_percent','ytm_percent','ytc_percent','notes_symbols']

In [ ]:
df_int = df[interested_cols]

In [ ]:
df_int.info()

In [ ]:
sections = {
    "EQUITY & EQUITY RELATED": {
        "sublist": {
            "a) Listed/awaiting listing on Stock Exchanges": "listed_on_exchange",
            "b) Unlisted": "unlisted_on_exchange",
            "c) Foreign Securities and /or overseas ETF": "listed_overseas"
        }
    },
    "DEBT INSTRUMENTS": {
        "sublist": {
            "a) Listed/awaiting listing on the stock exchanges": "listed_on_exchange",
            "b) Privately Placed/Unlisted": "unlisted_on_exchange",
            "c) Securitised Debt Instruments": "sdi_securities",
            "d) Central Government Securities": "central_govt_securities",
            "e) State Government Securities": "state_govt_securities"
        }
    },
    "MONEY MARKET INSTRUMENTS": {
        "sublist": {
            "a) Commercial Paper": "commercial_paper",
            "b) Certificate of Deposits": "certificate_of_deposits",
            "c) Treasury Bills": "treasury_bills",
            "d) Bills Re- Discounting": "bills_re_discounting",
            "e) STRIPS": "strips"
        }
    },
    "OTHERS": {
        "sublist": {
            "a) Mutual Fund Units / Exchange Traded Funds": "mf_or_exchange_traded_funds",
            "b) Alternative Investment Funds": "alt_investment_finds",
            "c) Gold": "gold",
            "d) Short Term Deposits": "short_term_deposits",
            "e) Term Deposits Placed as Margins": "term_deposits_placed_as_Margin",
            "f) TREPS / Reverse Repo Investments": "treps"
        }
    },
    "Other Current Assets / (Liabilities)": {
        "sublist": {}
    },
    "DERIVATIVES": {
        "sublist": {
            "Stock Futures": "stock_futures",
            "INTEREST RATE SWAPS": "interest_rate_swaps",
            "Index Futures": "index_futures"
        }
    }
}

In [ ]:
# core logic to create a KV data structure
ommit_rows = ['','Notes & Symbols :-','GRAND TOTAL (AUM)','Name of the Instrument','Derivatives Total','Total']
section_dataframes = {}
current_section = None
section_start = None

current_section_type = None
current_sub_section_type = None

for idx, row in df_int.iterrows():
    holdings = None
    
    if row[0] == 'Notes & Symbols :-':
        break;
    else:
        if row[0] in sections:
            if current_section_type != row[0]:
                current_section_type = row[0]
                current_sub_section_type = ''
        elif row[0] in sections[current_section_type]['sublist']:
            # current_sub_section_type = row[0]
            current_sub_section_type = sections[current_section_type]['sublist'][row[0]]
        elif row[0] not in ommit_rows :
            holdings = {
                 'inst_name':row['inst_name'] ,  
                 'isin':row['isin'],
                 'sector':row['sector'],
                 'quantity':row['quantity'],
                 #'quantity':row['quantity'].str.extract(r'\((\d+)\)')
                 'market_value_lakhs':row['market_value_lakhs'] if row['market_value_lakhs']!='NIL' else 0.0,
                 'aum_percent':row['aum_percent'],
                 # 'esg_score':row['esg_score'] if ,
                 'esg_score':row.get('esg_score','') or '',
                 'notes_symbols':row['notes_symbols'],
                 'ytm_percent':row['ytm_percent'],
                 'ytc_percent':row['ytc_percent']
            }
            section_dataframes.setdefault(current_section_type,{}).setdefault(current_sub_section_type,[]).append(holdings)
        
    

In [ ]:
# conversion of flat_rows to flat_df
flat_rows = []

for section_type, subsections in section_dataframes.items():
    for sub_section_type, holdings_list in subsections.items():
        for holding in holdings_list:
            flat_rows.append({
                'section_type': section_type,
                'sub_section_type': sub_section_type,
                **holding  # expands the holding dict into flat columns
            })


final_df = pd.DataFrame(flat_rows)

In [ ]:
from datetime import datetime

filename = "all-schemes-monthly-portfolio---as-on-28th-february-2025.xlsx"
match = re.search(r'(\d{1,2})(?:st|nd|rd|th)-([a-z]+)-(\d{4})', filename, re.IGNORECASE)

if match:
    day, month_str, year = match.groups()
    try:
        # Convert to proper datetime object (month needs formatting)
        dt = datetime.strptime(f"{day} {month_str} {year}", "%d %B %Y")
        print("Month:", dt.month)
        print("Year:", dt.year)
    except ValueError:
        print("Invalid date format in string.")
else:
    print("No valid date found in filename.")


In [ ]:
import numpy as np

# renaming sections in the final df

final_df['section_type'] = final_df['section_type'].replace(
    {
        'EQUITY & EQUITY RELATED': 'equities',
        'OTHERS': 'other',
        'Other Current Assets / (Liabilities)': 'assets_or_liabilities',
        'DERIVATIVES': 'derivatives',
        'MONEY MARKET INSTRUMENTS': 'money_market_inst'
    }
)

final_df['mf_fund_id'] = 123
final_df['mf_fund_code'] = sheet_name
final_df['fund_holder'] = 'SBI'

final_df=final_df.fillna(0.0)

# getting the month and year from file name 



## subsections
# final_df['sub_section_type'] = final_df['sub_section_type'].replace(
#     {
#         'a) Listed/awaiting listing on Stock Exchanges': 'listed_on_exchange',
#         'b) Unlisted': 'unlisted',
#         'c) Foreign Securities and /or overseas ETF': 'overseas_investments',
#         'e) TREPS / Reverse Repo Investments': 'treps_reserve_repo',
#         'Stock Futures':'stock_futures'
#     }
# )

#marking as asset or liability
final_df['type'] = np.where(
    (final_df['market_value_lakhs'] < 0 ),
    'liability',  # value if condition is True
    'asset'          # value if condition is False
)

In [ ]:
final_df

In [ ]:
selected_section = final_df[final_df['section_type']  == 'derivatives']

In [ ]:
selected_section

In [ ]:
# get the last details for bechmark and riskometer

tail = df.tail(30)
import re
benchmark_name = None
for _, row in tail.iterrows():
    row_text = " ".join(str(cell).strip() for cell in row if pd.notna(cell))
    
    # Extract benchmark name
    if "Benchmark Name :" in row_text:
        match = re.search(r"Benchmark Name\s*:\s*(.*)", row_text)
        if match:
            benchmark_name = match.group(1).strip()

In [ ]:
benchmark_name

In [ ]:
#  code to create numbers for portfolio asset allocation

In [ ]:
#  code create portfolio by industry allocation

In [ ]:
#@import parsing_test as pt

from parsing_test import generate_mf_ds

In [ ]:
import pandas as pd

# Path to your Excel file
file_path = "all-schemes-monthly-portfolio---as-on-28th-february-2025.xlsx"

# Load the workbook
xls = pd.ExcelFile(file_path)

In [ ]:
import parsing_test
import importlib

# Reload the module to pick up any changes
importlib.reload(parsing_test)

# Now call your function
parsed_df = parsing_test.generate_mf_ds(123,'SCF',xls,2,2025)

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas.io.excel._base import ExcelFile

mf_component_section_mapping = {
    "EQUITY & EQUITY RELATED": {
        "sublist": {
            "a) Listed/awaiting listing on Stock Exchanges": "listed_on_exchange",
            "b) Unlisted": "unlisted_on_exchange",
            "c) Foreign Securities and /or overseas ETF": "listed_overseas"
        }
    },
    "DEBT INSTRUMENTS": {
        "sublist": {
            "a) Listed/awaiting listing on the stock exchanges": "listed_on_exchange",
            "b) Privately Placed/Unlisted": "unlisted_on_exchange",
            "c) Securitised Debt Instruments": "sdi_securities",
            "d) Central Government Securities": "central_govt_securities",
            "e) State Government Securities": "state_govt_securities"
        }
    },
    "MONEY MARKET INSTRUMENTS": {
        "sublist": {
            "a) Commercial Paper": "commercial_paper",
            "b) Certificate of Deposits": "certificate_of_deposits",
            "c) Treasury Bills": "treasury_bills",
            "d) Bills Re- Discounting": "bills_re_discounting",
            "e) STRIPS": "strips"
        }
    },
    "OTHERS": {
        "sublist": {
            "a) Mutual Fund Units / Exchange Traded Funds": "mf_or_exchange_traded_funds",
            "b) Alternative Investment Funds": "alt_investment_finds",
            "c) Gold": "gold",
            "d) Short Term Deposits": "short_term_deposits",
            "e) Term Deposits Placed as Margins": "term_deposits_placed_as_Margin",
            "f) TREPS / Reverse Repo Investments": "treps"
        }
    },
    "Other Current Assets / (Liabilities)": {
        "sublist": {}
    },
    "DERIVATIVES": {
        "sublist": {
            "Stock Futures": "stock_futures",
            "INTEREST RATE SWAPS": "interest_rate_swaps",
            "Index Futures": "index_futures"
        }
    }
}


def read_sheet_for_mf(sheet_name, xls_file):
    mf_df = xls_file.parse(sheet_name, header=5)
    mf_df.dropna(how='all',
                 inplace=True)  # drop completly blank rows , make changes directly to the df -- inplace = true
    mf_df = mf_df.loc[:, ~mf_df.columns.str.contains('BRSR')]  # not interested in any BRSR col
    print('1')
    mf_df.info()
    return mf_df


def col_rename(df: DataFrame):
    df.columns = df.columns.str.replace(r'Name of.*', 'inst_name', regex=True)
    df.columns = df.columns.str.replace(r'Rating.*', 'sector', regex=True)
    df.columns = df.columns.str.replace(r'Market[\s\S]*', 'market_value_lakhs', regex=True)
    df.columns = df.columns.str.replace(r'YTM.*', 'ytm_percent', regex=True)
    df.columns = df.columns.str.replace(r'YTC.*', 'ytc_percent', regex=True)
    df.columns = df.columns.str.replace(r'Notes.*', 'notes_symbols', regex=True)
    df.rename(columns={
        "ISIN": "isin",
        "Quantity": "quantity",
        "% to AUM": "aum_percent",
    })

    return df


def generate_holdings_map_ds(df: DataFrame):
    ommit_rows = ['', 'Notes & Symbols :-', 'GRAND TOTAL (AUM)', 'Name of the Instrument', 'Derivatives Total', 'Total']
    section_dataframes = {}

    current_section_type = None
    current_sub_section_type = None

    for idx, row in df.iterrows():
        if row[0] == 'Notes & Symbols :-':
            break;
        else:
            if row[0] in mf_component_section_mapping:
                if current_section_type != row[0]:
                    current_section_type = row[0]
                    current_sub_section_type = ''
            elif row[0] in mf_component_section_mapping[current_section_type]['sublist']:
                # current_sub_section_type = row[0]
                current_sub_section_type = mf_component_section_mapping[current_section_type]['sublist'][row[0]]
            elif row[0] not in ommit_rows:
                holdings = {
                    'inst_name': row['inst_name'],
                    'isin': row['isin'],
                    'sector': row['sector'],
                    'quantity': row['quantity'],
                    # 'quantity':row['quantity'].str.extract(r'\((\d+)\)')
                    'market_value_lakhs': row['market_value_lakhs'] if row['market_value_lakhs'] != 'NIL' else 0.0,
                    'aum_percent': row['aum_percent'],
                    # 'esg_score':row['esg_score'] if ,
                    'esg_score': row.get('esg_score', '') or '',
                    'notes_symbols': row['notes_symbols'],
                    'ytm_percent': row['ytm_percent'],
                    'ytc_percent': row['ytc_percent']
                }
                section_dataframes.setdefault(current_section_type, {}).setdefault(current_sub_section_type, []).append(
                    holdings)

        return section_dataframes


def convert_ds_to_df(ds_map):
    flat_rows = []

    for section_type, subsections in ds_map.items():
        for sub_section_type, holdings_list in subsections.items():
            for holding in holdings_list:
                flat_rows.append({
                    'section_type': section_type,
                    'sub_section_type': sub_section_type,
                    **holding  # expands the holding dict into flat columns
                })

    return pd.DataFrame(flat_rows)


def post_process(df: DataFrame, mf_fund_id: int, mf_fund_code: str, disclose_month: int, disclose_year: int):
    # rename the section types
    df['section_type'] = df['section_type'].replace(
        {
            'EQUITY & EQUITY RELATED': 'equities',
            'OTHERS': 'other',
            'Other Current Assets / (Liabilities)': 'assets_or_liabilities',
            'DERIVATIVES': 'derivatives',
            'MONEY MARKET INSTRUMENTS': 'money_market_inst'
        }
    )

    df['mf_fund_id'] = mf_fund_id
    df['mf_fund_code'] = mf_fund_code
    df['fund_holder'] = 'SBI'

    # remove all NaN

    df = df.fillna(0.0)

    # marking as asset or liability
    df['type'] = np.where(
        (df['market_value_lakhs'] < 0),
        'liability',  # value if condition is True
        'asset'  # value if condition is False
    )

    # populate disclosure month and year
    df['disclose_month'] = disclose_month
    df['disclose_year'] = disclose_year

    return df


def generate_mf_ds(mf_code_id: int, mf_code_str: str, xls_file_fp: ExcelFile, disclosure_month: int,
                   disclosure_year: int):
    # read the sheet
    mf_df = read_sheet_for_mf(sheet_name=mf_code_str, xls_file=xls_file_fp)

    # rename cols
    mf_df = col_rename(df=mf_df)

    interested_cols = ['inst_name', 'isin', 'sector', 'quantity', 'market_value_lakhs', 'aum_percent', 'ytm_percent',
                       'ytc_percent', 'notes_symbols']

    # pick interested cols
    df_int = mf_df[interested_cols]

    # generate intermediate ds
    holdings_ds = generate_holdings_map_ds(df_int)

    # convert to dataframe
    holdings_df = convert_ds_to_df(holdings_ds)

    # post process
    holdings_df = post_process(df=holdings_df, mf_fund_id=mf_code_id, mf_fund_code=mf_code_str,
                               disclose_month=disclosure_month,
                               disclose_year=disclosure_year)

    return holdings_df


In [ ]:
import pandas as pd

# Path to your Excel file
file_path = "all-schemes-monthly-portfolio---as-on-28th-february-2025.xlsx"

# Load the workbook
xls = pd.ExcelFile(file_path)

In [ ]:
tested = generate_mf_ds(mf_code_id=123,mf_code_str='SMEEF',xls_file_fp=xls,disclosure_month=2,disclosure_year=2025)

In [ ]:
# testing for GCP 

In [ ]:
gcs_path = 'gs://projx-tb-extracts-staging/pxsbi/all-schemes-monthly-portfolio---as-on-28th-february-2025.xlsx'

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd 
xls_file = pd.read_excel(gcs_path, engine='openpyxl')